In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('duplicate_question_pair_train.csv'
                 )

In [3]:
new_df = df.sample(100000,random_state=2)
new_df.shape

(100000, 6)

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate
from tensorflow.keras.models import Model

# Replace NaN values with an empty string and ensure all values are strings
new_df['question1'] = new_df['question1'].fillna('').astype(str)
new_df['question2'] = new_df['question2'].fillna('').astype(str)

# Initialize and fit tokenizer on both question columns
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(new_df['question1']) + list(new_df['question2']))

# Determine vocabulary size
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding
print("Vocabulary size:", vocab_size)

# Convert questions to sequences
q1_seq = tokenizer.texts_to_sequences(new_df['question1'])
q2_seq = tokenizer.texts_to_sequences(new_df['question2'])

# Find max sequence length and pad sequences
max_seq_length = max(max(len(seq) for seq in q1_seq), max(len(seq) for seq in q2_seq))
padded_q1_seq = pad_sequences(q1_seq, maxlen=max_seq_length)
padded_q2_seq = pad_sequences(q2_seq, maxlen=max_seq_length)




Vocabulary size: 49529


In [5]:
padded_q1_seq.shape

(100000, 237)

In [6]:
print(tokenizer.word_index)

{'the': 1, 'what': 2, 'is': 3, 'how': 4, 'i': 5, 'a': 6, 'to': 7, 'in': 8, 'do': 9, 'of': 10, 'are': 11, 'and': 12, 'can': 13, 'for': 14, 'you': 15, 'why': 16, 'my': 17, 'best': 18, 'it': 19, 'on': 20, 'does': 21, 'or': 22, 'if': 23, 'be': 24, 'which': 25, 'some': 26, 'that': 27, 'have': 28, 'get': 29, 'with': 30, 'should': 31, 'an': 32, 'from': 33, 'your': 34, 'india': 35, 'will': 36, 'when': 37, 'people': 38, 'like': 39, 'who': 40, 'at': 41, 'good': 42, 'would': 43, 'there': 44, 'as': 45, 'about': 46, 'not': 47, 'between': 48, 'one': 49, 'most': 50, 'we': 51, 'quora': 52, 'way': 53, 'make': 54, 'did': 55, 'by': 56, 'where': 57, 'any': 58, 'was': 59, 'life': 60, 'me': 61, 'so': 62, 'after': 63, 'time': 64, 'they': 65, 'this': 66, 'money': 67, 'know': 68, 'difference': 69, 'has': 70, 'learn': 71, "what's": 72, 'use': 73, 'new': 74, 'am': 75, 'think': 76, 'their': 77, 'much': 78, 'many': 79, 'work': 80, 'all': 81, 'than': 82, 'indian': 83, 'find': 84, 'more': 85, 'someone': 86, 'ever': 

In [7]:
vectorizer_features_and_manual_features = pd.read_csv("vectorizer_features_and_manual_features.csv")

In [8]:
vectorizer_features_and_manual_features.shape

(100000, 3022)

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np

# Load manually calculated features from CSV file
manual_features_df = pd.read_csv('vectorizer_features_and_manual_features.csv')
manual_features_array = manual_features_df.to_numpy()

# Combine question sequences
padded_combined_seq = np.hstack((padded_q1_seq, padded_q2_seq))

# Define model inputs
sequence_input = Input(shape=(padded_combined_seq.shape[1],), name='sequence_input')
x = Embedding(input_dim=vocab_size, output_dim=128)(sequence_input)
x = LSTM(64, return_sequences=True, kernel_regularizer=l2(0.01))(x)  # First LSTM layer
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

# Add second LSTM layer
x = LSTM(64, return_sequences=False, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

# Manually calculated features input
manual_features_input = Input(shape=(manual_features_array.shape[1],), name='manual_features_input')
y = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(manual_features_input)
y = BatchNormalization()(y)
y = Dropout(0.3)(y)

# Add another dense layer for manually calculated features
y = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(y)
y = BatchNormalization()(y)
y = Dropout(0.3)(y)

# Concatenate both outputs
combined = Concatenate()([x, y])

# Add multiple dense layers for the combined output
combined = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(combined)
combined = BatchNormalization()(combined)
combined = Dropout(0.3)(combined)

combined = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(combined)
combined = BatchNormalization()(combined)
combined = Dropout(0.3)(combined)

output = Dense(1, activation='sigmoid')(combined)

# Create and compile the model
model = Model(inputs=[sequence_input, manual_features_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
# early_stopping = EarlyStopping(
#     monitor='val_loss', 
#     patience=3,  
#     restore_best_weights=True  
# )

# Train the model
y_train = new_df['is_duplicate'].values  # Assuming target labels are in 'is_duplicate'
history = model.fit(
    [padded_combined_seq, manual_features_array],
    y_train,
    epochs=15,
    batch_size=32,
    validation_split=0.2,
    # callbacks=[early_stopping]  # Enable early stopping callback
)


Epoch 1/15
2500/2500 [==============================] - 1159s 462ms/step - loss: 1.1891 - accuracy: 0.7161 - val_loss: 0.6145 - val_accuracy: 0.6773
Epoch 2/15
2500/2500 [==============================] - 1171s 468ms/step - loss: 0.4893 - accuracy: 0.7797 - val_loss: 0.5146 - val_accuracy: 0.7466
Epoch 3/15
2500/2500 [==============================] - 1168s 467ms/step - loss: 0.4593 - accuracy: 0.8000 - val_loss: 0.7133 - val_accuracy: 0.6464
Epoch 4/15
2500/2500 [==============================] - 1161s 465ms/step - loss: 0.4326 - accuracy: 0.8185 - val_loss: 0.5607 - val_accuracy: 0.7232
Epoch 5/15
2500/2500 [==============================] - 1162s 465ms/step - loss: 0.4106 - accuracy: 0.8298 - val_loss: 0.8289 - val_accuracy: 0.6640
Epoch 6/15
2500/2500 [==============================] - 1153s 461ms/step - loss: 0.3936 - accuracy: 0.8380 - val_loss: 0.6801 - val_accuracy: 0.7023
Epoch 7/15
2500/2500 [==============================] - 1189s 476ms/step - loss: 0.3830 - accuracy: 0.8421

In [10]:
model.save('LSTM4')

INFO:tensorflow:Assets written to: LSTM4\assets


INFO:tensorflow:Assets written to: LSTM4\assets


In [11]:
# Save the tokenizer to a file
import pickle
with open('tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)